In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

ImportError: DLL load failed: 找不到指定的模块。

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified


### SGD

In [14]:
# loss func
criterion = nn.CrossEntropyLoss()

# models and optims
optim_name = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad']
model_list = []
for i in range(len(optim_name)):
    model_list.append(Net())

optim_list = []
# SGD
optim_SGD = optim.SGD(model_list[0].parameters(), lr=0.001)
optim_list.append(optim_SGD)
# Adam
optim_Adam = optim.Adam(model_list[1].parameters(), lr=0.001)
optim_list.append(optim_Adam)
# RMSProp
optim_RMSprop = optim.RMSprop(model_list[2].parameters(), lr=0.001)
optim_list.append(optim_RMSprop)
# Adadelta
optim_Adadelta = optim.Adadelta(model_list[3].parameters(), lr=0.001)
optim_list.append(optim_Adadelta)
# Adagrad
optim_Adagrad = optim.Adagrad(model_list[4].parameters(), lr=0.001) 
optim_list.append(optim_Adagrad)


### Adam

In [17]:
epoch_num = 20
loss_matrix = np.zeros((len(optim_name), epoch_num))

In [21]:
for epoch in range(epoch_num):  # loop over the dataset multiple times
    iter_num = 0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        iter_num = i
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()
#         # forward + backward + optimize
#         outputs = net(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

        # 同步训练所有模型
        for j in range(len(optim_name)):
            optim_list[j].zero_grad()
            outputs = model_list[j](inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optim_list[j].step()
            loss_matrix[j, epoch] += loss.item()
                   
        #print('[epoch:%d, iter:%5d] loss: %.3f optim: %s' %
        #          (epoch + 1, i + 1, loss.item(), optim_name[j]))
    loss_matrix[:, epoch] /= iter_num
    print("[epoch:{}]".format(epoch+1))
    for idx in range(len(optim_name)):
        print("     loss:{}, optim:{}".format(loss_matrix[idx, epoch], optim_name[idx]))
   
print('Finished Training')

[epoch:1, iter: 2000] loss: 2.314 optim: SGD
[epoch:1, iter: 2000] loss: 2.781 optim: Adam
[epoch:1, iter: 2000] loss: 2.160 optim: RMSprop
[epoch:1, iter: 2000] loss: 2.313 optim: Adadelta
[epoch:1, iter: 2000] loss: 2.303 optim: Adagrad
[epoch:1, iter: 4000] loss: 2.330 optim: SGD
[epoch:1, iter: 4000] loss: 2.423 optim: Adam
[epoch:1, iter: 4000] loss: 2.823 optim: RMSprop
[epoch:1, iter: 4000] loss: 2.210 optim: Adadelta
[epoch:1, iter: 4000] loss: 2.162 optim: Adagrad
[epoch:1, iter: 6000] loss: 2.284 optim: SGD
[epoch:1, iter: 6000] loss: 2.360 optim: Adam
[epoch:1, iter: 6000] loss: 1.764 optim: RMSprop
[epoch:1, iter: 6000] loss: 2.286 optim: Adadelta
[epoch:1, iter: 6000] loss: 1.534 optim: Adagrad
[epoch:1, iter: 8000] loss: 2.277 optim: SGD
[epoch:1, iter: 8000] loss: 1.150 optim: Adam
[epoch:1, iter: 8000] loss: 1.357 optim: RMSprop
[epoch:1, iter: 8000] loss: 2.303 optim: Adadelta
[epoch:1, iter: 8000] loss: 1.689 optim: Adagrad
[epoch:1, iter:10000] loss: 2.219 optim: SGD

KeyboardInterrupt: 